In [1]:
from game_logic.ghostAgents import RandomGhost
from entregables.maxNAgent import MaxNAgent
from game_logic.randomPacman import RandomPacman
from game_logic.PacmanEnvAbs import PacmanEnvAbs
import random
import math
import numpy as np
from game_logic.game_util import process_state
from main import get_default_agents
import timeit 
import pandas as pd

all_layouts = [
        "custom1",
        "custom2",
        "capsuleClassic",
        "contestClassic",
        "mediumClassic",
        "minimaxClassic",
        "openClassic",
        "originalClassic",
        "smallClassic",
        "testClassic",
        "trappedClassic",
        "trickyClassic",
        "mediumGrid",
        "smallGrid"
    ]

# Parametros para un test
class TestParams():
    def __init__(self, test_name, layout, pacman_agent, ghost_agent_0, ghost_agent_1):        
        self.test_name = test_name
        self.layout = layout
        self.pacman_agent = pacman_agent
        self.ghost_agent_0 = ghost_agent_0
        self.ghost_agent_1 = ghost_agent_1


In [2]:
def run_test(test_params):  
    t0 = timeit.default_timer()   
    pacman_agent = test_params.pacman_agent
    ghost_agent_0 = test_params.ghost_agent_0
    ghost_agent_1 = test_params.ghost_agent_1
    agents = [pacman_agent, ghost_agent_0, ghost_agent_1]
    agents.extend(get_default_agents(3, 10))    
    done = False
    env = PacmanEnvAbs(agents = agents, view_distance = (2, 2))      
    game_state = env.reset(enable_render= False, layout_name= test_params.layout)
    turn_index = 0
    while (not(done)): # jugar   
        action = agents[turn_index].getAction(game_state)
        game_state, rewards, done, info = env.step(action, turn_index)        
        turn_index = (turn_index + 1) % env._get_num_agents()  
    t1 = timeit.default_timer()    
    time = t1-t0  
    assert(game_state.getNumAgents()>=2) # que el juego tenga mas de 2 agentes
    if game_state.getNumAgents()==2: # vector de rewards con los primeros 2 rewards y nan
        ret = game_state.get_rewards()
        ret.append(np.nan)
    else: # vector de rewards con los primeros 3 rewards
        ret = game_state.get_rewards()[0:3]
    return ret, time

In [3]:
pacman_agent = RandomPacman(index = 0)
ghost_agent_0 = MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=12, number_of_unrolls=6)
ghost_agent_1 = RandomGhost(index = 2)
sample_test = TestParams("PrimerTest", "mediumGrid", pacman_agent, ghost_agent_0, ghost_agent_1)
print(run_test(sample_test))


([-1498, -1175.0, nan], 0.5497670379999988)


In [4]:
selected_layouts = [  
        "originalClassic",  
        "mediumGrid",
        "smallGrid"
    ]
# 2 pacmans random y MaxN-MCTS
pacmans = [("rnd_pcmn",RandomPacman(index = 0)), ("MAXN_pcmn",MaxNAgent(index = 0, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10))]
# 2 ghosts 0 MC y MCTS
ghosts0s = [
    ("MAXN_MC_gh0",MaxNAgent(index = 1, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10)),
    ("MAXN_MCTS_gh0",MaxNAgent(index = 1, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10)),
]
# 2 ghosts 2 MC y MCTS
ghosts1s = [
    ("MAXN_MC_gh1",MaxNAgent(index = 2, unroll_type="MC", max_unroll_depth=5, number_of_unrolls=10)),
    ("MAXN_MCTS_gh1",MaxNAgent(index = 2, unroll_type="MCTS", max_unroll_depth=5, number_of_unrolls=10)),
]

In [5]:
all_tests = []
for layout in selected_layouts:
    for pacman_name, pacman_agent in pacmans:
        for ghost0_name, ghost_0 in ghosts0s:
            for ghost1_name, ghost_1 in ghosts1s:
                name = layout+"_"+pacman_name+"_"+ghost0_name+"_"+ghost1_name
                test = TestParams(name, layout, pacman_agent, ghost_agent_0, ghost_agent_1)
                all_tests.append(test)
number_of_runs = 10

In [6]:
results = []
t0 = timeit.default_timer()   
for i in range(len(all_tests)):   
        values = []
        times = []    
        for _ in range(number_of_runs):
            value, time = run_test(all_tests[i])            
            values.append(value)
            times.append(time)
        #compute avg and std and and add to results
        value_avg = np.round(np.mean(np.array(values), axis=0),2)
        time_avg = np.round(np.mean(np.array(times)),2)
        value_std = np.round(np.std(np.array(values), axis=0),2)
        time_std = np.round(np.std(np.array(times)),2)
        result = {'TestName':all_tests[i].test_name,'Time_Avg':time_avg,'Time_Std':time_std, 
                  'P_Value_Avg':value_avg[0], 
                  'P_Value_Std':value_std[0],
                  'GH0_Value_Avg':value_avg[1], 
                  'GH0_Value_Std':value_std[1],
                  'GH1_Value_Avg':value_avg[2], 
                  'GH1_Value_Std':value_std[2]
                  }
        results.append(result)
results = pd.DataFrame(results)
t1 = timeit.default_timer()    
time = t1-t0  
print("All tests time: ", time, "seconds.")    
display(results)

All tests time:  5227.01983676 seconds.


,TestName,Time_Avg,Time_Std,P_Value_Avg,P_Value_Std,GH0_Value_Avg,GH0_Value_Std,GH1_Value_Avg,GH1_Value_Std
0,originalClassic_rnd_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,8.78,4.51,-1447.9,43.23,-2858.1,1482.39,-2157.3,1239.96
1,originalClassic_rnd_pcmn_MAXN_MC_gh0_MAXN_MCTS...,15.78,7.46,-1466.3,49.60,-4785.6,2333.75,-4048.8,2004.38
2,originalClassic_rnd_pcmn_MAXN_MCTS_gh0_MAXN_MC...,10.68,4.50,-1439.2,47.08,-3451.3,1466.87,-2910.4,1337.95
3,originalClassic_rnd_pcmn_MAXN_MCTS_gh0_MAXN_MC...,11.74,8.93,-1438.1,104.72,-3818.3,2921.90,-3056.5,2722.23
4,originalClassic_MAXN_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,122.25,48.20,-1532.4,266.77,-35507.6,14383.20,-46518.3,18713.68
5,originalClassic_MAXN_pcmn_MAXN_MC_gh0_MAXN_MCT...,48.35,34.54,-1490.5,118.80,-13881.4,10308.93,-17596.0,13488.12
6,originalClassic_MAXN_pcmn_MAXN_MCTS_gh0_MAXN_M...,87.89,56.94,-1509.1,185.81,-23642.0,15086.38,-30682.3,19962.06
7,originalClassic_MAXN_pcmn_MAXN_MCTS_gh0_MAXN_M...,108.83,67.84,-1562.6,124.97,-29751.5,18884.77,-38896.4,24936.04
8,mediumGrid_rnd_pcmn_MAXN_MC_gh0_MAXN_MC_gh1,5.33,5.61,-1356.3,615.17,-21772.5,23951.22,NaN,NaN
9,mediumGrid_rnd_pcmn_MAXN_MC_gh0_MAXN_MCTS_gh1,6.20,4.69,-1365.5,557.84,-26540.5,20733.27,NaN,NaN
